Process of JPEG:

1. DCT: Discrete cosine transform, spatial domain >> frequency domain

2. Quantization:
    The human eye isn't good at distinguishing a high frequency (rapidly varying) brightness variation. Thus, we simply 
    dividing each component in the frequency domain by a constant for that component, and then rounding to the nearest 
    integer. Many of the higher frequency components are rounded to zero, and many of the rest become small positive or 
    negative numbers.
    
3. Zigzag scan: reshape to a 1D array

4. Lossless encoding: Run-length encoding, the same value are stored as a single value and count.

5. Huffman encoding:
    Finding a code in linear time to the number of input weights if these weights are sorted. A Huffman tree that omits 
    unused symbols produces the most optimal code lengths.

6. Huffman decoding

7. Lossless decoding

8. Inverse-zigzag scan

9. Inverse quantization

10. Inverse DCT

In [39]:
%matplotlib inline
import numpy as np
import math
import matplotlib.pyplot as plt

In [40]:
q_mat = np.array([[16, 11, 10, 16, 24, 40, 51, 61],
                  [12, 12, 14, 19, 26, 58, 60, 55],
                  [14, 13, 16, 24, 40, 57, 69, 56],
                  [14, 17, 22, 29, 51, 87, 80, 62],
                  [18, 22, 37, 56, 68, 109, 103, 77],
                  [24, 35, 55, 64, 81, 104, 113, 92],
                  [49, 64, 78, 87, 103, 121, 120, 101],
                  [72, 92, 95, 98, 112, 100, 103, 99]])

def dct(image_mat):
    '''Discrete cosine transform'''
    N = 8
    coeff_mat = np.zeros((N, N), dtype=int)
    C = [1.0/math.sqrt(2.0)] + [1.0] * (N-1)
    for row in range(N):
        for col in range(N):
            for x in range(N):
                for y in range(N):
                    coeff_mat[row][col] = int(1/4  * C[row] * C[col] *
                    sum(image_mat[x][y] * math.cos(math.pi*(2*x+1)*row/16) *
                    math.cos(math.pi*(2*y+1)*col/16)))
    return coeff_mat

def idct(coeff_mat):
    '''Inverse Discrete cosine transform'''
    N = 8
    image_mat = np.zeros((N, N), dtype=int)
    C = [1.0/math.sqrt(2.0)] + [1.0] * (N-1)
    for x in range(N):
        for y in range(N):
            image_mat[x][y] = int (1/4 *
                                   sum([C[row] * C[col] *
                                        coeff_mat[row][col] *
                                        math.cos(math.pi*(2*x+1)*row/16) *
                                        math.cos(math.pi*(2*y+1)*col/16) 
                                        for row in range(N) for col in range(N)]))
    return image_mat

def q(coeff_mat, q_mat):
    '''Quantization'''
    N = 8
    q_coeff_mat = np.zeros((N, N), dtype=coeff_mat.dtype)
    for i in range(N):
        for j in range(N):
            q_coeff_mat[i, j] = int(round(coeff_mat[i, j] / q_mat[i, j]))
    return q_coeff_mat

def iq(q_coeff_mat, q_mat):
    '''Inverse Quantization'''
    return (q_coeff_mat * q_mat)

def zigzag(coeff_mat):
    N = 8
    coeff_list = []
    for i in range(N):
        if i%2 == 0:
            x, y = i, 0
            for k in range(i+1):
                coeff_list.append(coeff_mat[x,y])
                x, y = x-1, y+1
        else:
            x, y = 0, i
            for k in range(i+1):
                coeff_list.append(coeff_mat[x,y])
                x, y = x+1, y-1
    for j in range(1, N):
        if j%2 != 0:
            x, y = N-1, j
            for k in range(N-j):
                coeff_list.append(coeff_mat[x,y])
                x, y = x-1, y+1
        else:
            x, y = j, N-1
            for k in range(N-j):
                coeff_list.append(coeff_mat[x,y])
                x, y = x+1, y-1
    return coeff_list

def izigzag(input_coeff_list):
    N = 8
    coeff_list = input_coeff_list[:]
    coeff_mat = np.zeros((N, N), coeff_list[0].dtype)
    for i in range(N):
        if i%2 == 0:
            x, y = i, 0
            for k in range(i+1):
                coeff_mat[x,y] = coeff_list.pop(0)
                x, y = x-1, y+1
        else:
            x, y = 0, i
            for k in range(i+1):
                coeff_mat[x,y] = coeff_list.pop(0)
                x, y = x+1, y-1
    for j in range(1, N):
        if j%2 != 0:
            x, y = N-1, j
            for k in range(N-j):
                coeff_mat[x,y] = coeff_list.pop(0)
                x, y = x-1, y+1
        else:
            x, y = j, N-1
            for k in range(N-j):
                coeff_mat[x,y] = coeff_list.pop(0)
                x, y = x+1, y-1
    return coeff_mat

def rl_enc(coeff_list):
    '''Run Length Encoding'''
    rl_list = []
    i = 0
    zero_count = 0
    while i < len(coeff_list):
        if coeff_list[i] != 0:
            rl_list.append((zero_count, coeff_list[i]))
            i += 1
            zero_count = 0
        else:
            i += 1
            zero_count += 1
    rl_list.append((0, 0))  
    return rl_list

def rl_dec(rl_list):
    '''Run Length Decoding'''
    coeff_list = []
    for t in rl_list[:-1]:
        for i in range(t[0]):
            coeff_list.append(0)
        coeff_list.append(t[1])        
    coeff_list = coeff_list + [0]*(8*8-1 - len(coeff_list))
    return coeff_list

Here, we'll slice the picture to many 8*8 blocks.
Afterthat, we'll apply each block with mentioned jpeg process.
This is a devide-and-conquer technique.

In [41]:
# Key for sorting a list of tuples
def getKey(item):
    return item[0]

# Construct a huffman tree according to the frequency of chars of the text
def gen_huffman_tree(text):
    text_list = [c for c in text]
    alphabet = set(text_list)
    h_tree = [ (text_list.count(c), (c, ) ) for c in alphabet ]
    while len(h_tree) >= 2:
        h_tree = sorted(h_tree, key=getKey)
        h_tree = [ (h_tree[0][0] + h_tree[1][0], (h_tree[0], h_tree[1])) ] + h_tree[2:]
    return h_tree

# Build Huffman dictionary out of a Huffman tree
def gen_huffman_dict(h_tree_node, code, h_dict):
    if len(h_tree_node[1]) == 1:
        h_dict[h_tree_node[1][0]] = code
    else:
        gen_huffman_dict(h_tree_node[1][0], code+'0', h_dict)
        gen_huffman_dict(h_tree_node[1][1], code+'1', h_dict)
    return

# Encode a text message according to a Huffman dictionary
def huffman_enc(h_dict, text):
    h_code = ''
    for c in text:
        h_code = h_code + h_dict[c]
    return h_code

# Decode a bit stream following a Huffman tree
def huffman_dec(h_tree, h_code):
    dec_text = ''
    i = 0
    while i < len(h_code):
        cur_node = h_tree[0]
        while len(cur_node[1]) == 2:
            cur_node = cur_node[1][0] if h_code[i] == '0' else cur_node[1][1]
            i += 1
        dec_text = dec_text + cur_node[1][0]           
    return dec_text

# Decode a bit stream following a Huffman tree
# Output a list
def huffman_dec_list(h_tree, h_code):
    dec_list = []
    i = 0
    while i < len(h_code):
        cur_node = h_tree[0]
        while len(cur_node[1]) == 2:
            cur_node = cur_node[1][0] if h_code[i] == '0' else cur_node[1][1]
            i += 1
        dec_list.append(cur_node[1][0])           
    return dec_list

In [ ]:
from scipy import misc
import matplotlib.pyplot as plt
import huffman as hf


image_src = misc.imread('lenna_grey.png')
H, W = image_src.shape
N = 8

run_pair_list = []  # To collect all run-length pairs of AC components
dc_list = []        # To collect all DC components
decode_dict= {}     # To store all intermediate results

# Encoding Phase
for r in range(0, H//N):
    for c in range(0, W//N):
        image_mat = image_src[r*N:(r+1)*N, c*N:(c+1)*N]
        coeff_mat = dct(image_mat)
        q_coeff_mat = q(coeff_mat, q_mat)
        zz = zigzag(q_coeff_mat)
        dc_list.append(zz[0])
        ac_run = rl_enc(zz[1:])
        decode_dict[(r, c)] = {'imag':image_mat,
                               'coeff_mat':coeff_mat,
                               'q_coeff_mat':q_coeff_mat,
                               'zz':zz,
                               'ac_run':ac_run}
        run_pair_list += ac_run    # Dump all ACs for tree Huffman Tree construction

# Building tree and dictionary for huffman encoding/decoding of DCs
dc_diff_list = dc_list[:]   # Make a deep copy
for i in range(1, len(dc_diff_list)):
    dc_diff_list[i] = dc_list[i-1] - dc_list[i]

dc_tree = hf.gen_huffman_tree(dc_diff_list)
dc_dict = {}
hf.gen_huffman_dict(dc_tree[0], '', dc_dict)
dc_bits = hf.huffman_enc(dc_dict, dc_diff_list)

dc_diff_list_dec = hf.huffman_dec_list(dc_tree, dc_bits)

dc_list_dec = dc_diff_list_dec[:]   # Make a deep copy
for i in range(1, len(dc_list_dec)):
    dc_list_dec[i] = dc_list_dec[i-1] - dc_diff_list_dec[i]

# Building tree and dictionary for huffman encoding/decoding of AC runs
rl_tree = hf.gen_huffman_tree(run_pair_list)
rl_dict = {}
hf.gen_huffman_dict(rl_tree[0], '', rl_dict)    


# Decoding Phase
for r in range(0, H//N):
    for c in range(0, W//N):
        ac_run = decode_dict[(r, c)]['ac_run']       # This is part of encoding
        run_bits = hf.huffman_enc(rl_dict, ac_run)   # Put here b/c we don't have huffman dict
                                                     # yet in the encoding phase
        
        ac_run_dec = hf.huffman_dec_list(rl_tree, run_bits)
        izz = [dc_list_dec[r*(W//N) + c]] +  rl_dec(ac_run_dec)
        iq_coeff_mat= izigzag(izz)
        icoeff_mat = iq(iq_coeff_mat, q_mat)
        image_dec = idct(icoeff_mat)
        
        decode_dict[(r, c)]['run_bits'] = run_bits
        decode_dict[(r, c)]['ac_run_dec'] = ac_run_dec
        decode_dict[(r, c)]['izz'] = izz
        decode_dict[(r, c)]['iq_coeff_mat'] = iq_coeff_mat
        decode_dict[(r, c)]['icoeff_mat'] = icoeff_mat
        decode_dict[(r, c)]['image_dec'] = image_dec
        
# Output decoded image
reconstructed = np.zeros((H, W), dtype=image_src.dtype)
for r in range(0, H//N):
    for c in range(0, W//N):
        reconstructed[r*N:(r+1)*N, c*N:(c+1)*N] = decode_dict[(r, c)]['image_dec']

plt.imshow(reconstructed)